# Build a Multi-Omics Container for GCN Training
The goal of this little notebook is to create one HDF5 container that contains a feature matrix consisting of our multi-omics data:
* Mutation Frequencies
* Gene Expression
* Methylation
We then can train our network using the iref PPI and these different data sources.

In [1]:
import h5py
import pandas as pd
import numpy as np

/home/sasse/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load the Data from three containers

In [9]:
# mutation frequencies
with h5py.File('../../data/pancancer/mutfreq_iref_unbalanced_syn.h5', 'r') as f:
    network = f['network'][:]
    mutfreq_features = f['features_mean'][:]
    #features_mean = f['features_mean'][:]
    node_names = f['gene_names'][:]
    y_train = f['y_train'][:]
    y_test = f['y_test'][:]
    if 'y_val' in f:
        y_val = f['y_val'][:]
    else:
        y_val = None
    train_mask = f['mask_train'][:]
    test_mask = f['mask_test'][:]
    if 'mask_val' in f:
        val_mask = f['mask_val'][:]
    else:
        val_mask = None

# methylation levels
with h5py.File('../../data/pancancer/iref_methylation_450k_unbalanced.h5', 'r') as f:
    meth_features = f['features'][:]

# gene expression
with h5py.File('../../data/pancancer/hotnet_iref_vec_input_unbalanced.h5', 'r') as f:
    ge_features = f['features'][:]

## Join Multi-Omics to form one feature matrix

In [13]:
multi_omics_features = np.concatenate((mutfreq_features, meth_features, ge_features), axis=1)
multi_omics_features.shape

(12129, 47)

## Write back everything to container

In [18]:
def write_hdf5_container(fname, adj, F, node_names, y_train, y_test, train_mask, test_mask):
    f = h5py.File(fname, 'w')
    string_dt = h5py.special_dtype(vlen=str)
    f.create_dataset('network', data=adj, shape=adj.shape)
    f.create_dataset('features', data=F, shape=F.shape)
    f.create_dataset('gene_names', data=node_names, dtype=string_dt)
    f.create_dataset('y_train', data=y_train, shape=y_train.shape)
    f.create_dataset('y_test', data=y_test, shape=y_test.shape)
    f.create_dataset('mask_train', data=train_mask, shape=train_mask.shape)
    f.create_dataset('mask_test', data=test_mask, shape=test_mask.shape)
    f.close()

    print ("Container written to {}".format(fname))

In [17]:
multi_omics_features = np.concatenate((mutfreq_features, meth_features, ge_features), axis=1)
write_hdf5_container('../../data/pancancer/iref_multiomics_unbal.h5', network,
                     multi_omics_features, node_names, y_train, y_test, train_mask, test_mask)

Container written to ../../data/pancancer/iref_multiomics_unbal.h5


## Compute Containers for combinations of two omics data

In [19]:
meth_ge_features = np.concatenate((meth_features, ge_features), axis=1)
write_hdf5_container('../../data/pancancer/iref_meth_and_ge.h5', network,
                     meth_ge_features, node_names, y_train, y_test, train_mask, test_mask)

Container written to ../../data/pancancer/iref_meth_and_ge.h5


In [20]:
meth_mutfreq_features = np.concatenate((meth_features, mutfreq_features), axis=1)
write_hdf5_container('../../data/pancancer/iref_meth_and_mutfreq.h5', network,
                     meth_mutfreq_features, node_names, y_train, y_test, train_mask, test_mask)

Container written to ../../data/pancancer/iref_meth_and_mutfreq.h5


In [21]:
mutfreq_ge_features = np.concatenate((mutfreq_features, ge_features), axis=1)
write_hdf5_container('../../data/pancancer/iref_mutfreq_and_ge.h5', network,
                     mutfreq_ge_features, node_names, y_train, y_test, train_mask, test_mask)

Container written to ../../data/pancancer/iref_mutfreq_and_ge.h5
